In [1]:
#Librerias necesarias
using Random #Para generar numero aleatorios
using CSV #Para trabajar con archivos .csv
using DataFrames #Para manejar datos en estructuras de dataframe

# Simulación de movimiento ABP

In [2]:
#Parametros de la simualcion
n = 400 #Numero de particulas
L = 100 #Largo de la caja #largo de lado de un cuadrado
Dr = 0.0225 #Coeficiente de difusion rotacional 
v0 = 30 #Velocidad de las particulas
dt = 0.001  #intervalo de tiempo
steps = 5000 #Pasos de tiempo
tend = steps * dt  #Tiempo total de la simulacion
a = 1 #Tamano de la particula
e = 3 #Epsilon #Constante
sigma = 2*a #Diametro de la particula 
r_min = 2^(1/6)*sigma #Distancia minima entre particulas en la fuerza de lennar jones
frec_save = 10 #Frecuencia de guardado de datos
incremento_rotacional = sqrt(2 * Dr * dt) 
#Angulos iniciales
theta = 2 * pi * rand(n) #angulos iniciales aleatorios en los que se mueven las particulas
cost = cos.(theta) 
sint = sin.(theta)
#Velocidades iniciales
velocidades = [v0 * cost, v0 * sint] #velocidades de todas las particulas

#Definicion de las posiciones iniciales
function posiciones_iniciales!(n, sigma, L)
    min = -L/2 + sigma #limites donde se puede colocar una particula
    max = L/2 - sigma #Se asegura de que esten dentro del area del cuadradao 

    posiciones = zeros(n, 2) #matriz de 0 de matano n x 2 para alamacenar las posiciones (x, y)
    X = 0 #se inician las coordenadas
    Y = 0

    for i in 1:n #loop que itera para cada particula
        valid_posicion = false #se inicia esta variable para validar si una particula esta en una posicion valida (que no se este sobreponioentdo)
        while  !valid_posicion # "esto estar apasando hasta que la posicion sea valida (True)"
            X = (max - min) * rand() + min #genera una posicion aleatoria dentre de los limin min y max
            Y = (max - min) * rand() + min 
            valid_posicion = true #esto asime que es valida hasta que se demuestre que no
            
            for j in 1:i #loop que verifica si la posicion esta a la distancia valida de las otras
                distance = sqrt((X - posiciones[j, 1])^2 + (Y - posiciones[j, 2])^2) #Calcula la distancia eclidianda entre la nueva posicion y las ya generadas
                if distance < sigma*2.0 #si la distanica es menor (al doble del diamtero de la particula) la posicion no es valida
                    valid_posicion = false #si no es valida rompe el bucle 
                    break
                end
            end
        end
        
        posiciones[i,1] = X  #cuando encuentra una posicion valida, la guarda en la matriz
        posiciones[i,2] = Y
    end 
    return posiciones 
end

# función que calcula la fuerza entre particulas utilizando el potencial de Lennard - Jones
function LJ_fuerza!(e, sigma, X, Y, L, r_min) 

    f_x = zeros(n) #iniciar los vectores donde se guardan los componentes de la fuerza en cada particula
    f_y = zeros(n)

    for i in 1:n, j in i+1:n #loop que itera sobre cada par de particular (i, j), cada par se considera solo una vez

        dx, dy = X[i] - X[j], Y[i] - Y[j] #calcula la diferencia de distnaica entre las coordenadas de cada una de las particulas del par
        dx -= L * round(dx / L) #se aplican las condiciones periodicas y busca la distancia mas corda cin particulas que puedan estar en el borde
        dy -= L * round(dy / L) #busca la distancia mas corta entre particulas 
        distancia = sqrt(dx^2 + dy^2) #calucla la distanica entre particulas

        if distancia < r_min #verifica si la distancia entre particulas es menor que r_min, se calucla para particuals que esten lo suficientemente cerca
            force = 4 * e * (12 * (sigma/distancia)^12 - 6 * (sigma/distancia)^6) / distancia^2 #se calcula la fuerza
            f_x[i] += force * dx #se agrega los componentes de x e y de la fuerza para las pares de particulas #tercera ley de nuewton
            f_y[i] += force * dy
            f_x[j] -= force * dx
            f_y[j] -= force * dy
        end
    end

    return f_x, f_y
end

# Funcion principal de la dinamica de movimiento dinamica_ABP
function dinamica_ABP!(X, Y, velocidades)
    
    #Cambio en el angulo de las particulas
    @. theta += incremento_rotacional * randn()  
    #Angulos actuales de las particulas, incremente en angulo por el coeficiente, y un numero aleatorio con distribucion normal


    #Actualizacion de las velocidades basado en el angulo de la particula
    velocidades[1] .= v0 * cos.(theta) #velocidad constante
    velocidades[2] .= v0 * sin.(theta) #Element wise
    #se recalculan en cada paso de tiempo en funcion del angulo theta actualizado #reflehja elc ambio de direccion
    
    #Calculo de LJ_fuerza
    f_x, f_y = LJ_fuerza!(e, sigma, X, Y, L, r_min)


    #Actualizacion de posiciones
    @. X += velocidades[1] * dt + f_x * dt # la nueva posicione el producto de la velocidad y la fuerza y el paso de tiempo
    @. Y += velocidades[2] * dt + f_y * dt #como afecta en el siguiente paso de tiempo
    

    #Condiciones de borde periodicas #residuo de la division
    @. X = mod(X + L / 2, L) - L / 2 #si sale la particula entra por el lado opuesto dentro del rango 
    @. Y = mod(Y + L / 2, L) - L / 2
    
    return X, Y
end

posiciones = posiciones_iniciales!(n, sigma, L)
X = posiciones[:, 1] #se extraen las posicionex #todas las coornadas de x de las particulas en la simulacion
Y = posiciones[:, 2]


posiciones_x = Vector{Vector{Float64}}() #vectores vacios el que tendra las coordenadas de las posiciones actualizadas
posiciones_y = Vector{Vector{Float64}}()

@time for i in 1:steps #loop que se ejecuta en cada Steps 
    dinamica_ABP!(X, Y, velocidades) #en cada paso de tiempo de tiempo se ejecuta la funcion que cambia las posiciones de las particulas
    if i == 1 || mod(i, frec_save) == 0 #solo se guardan cada tanto posiciones de las particulas #por tiempos #|| es or #multiplos de 10
        push!(posiciones_x, copy(X)) #es como append en python
        push!(posiciones_y, copy(Y))
    end
end

CSV.write("posiciones_particulas.csv", DataFrame(X = posiciones_x, Y = posiciones_y))

2-element Vector{Vector{Float64}}:
 [18.5900180657153, 29.98418507162129, 25.236271454515606, -29.574169754836692, 22.936681686305928, -29.627365229533634, -26.467110022158515, 22.67954683144095, -8.98875074123433, 2.1118025106761427  …  -28.883029573152527, 17.268205172360947, 28.217811019702424, -20.543575196628296, 12.955669107998737, -8.261049006019986, -27.96813081017157, 25.711116842722586, 10.459477105477857, -10.007650349444084]
 [-23.545938679873835, 0.9739843893835793, 16.22130090572272, 5.036713542787875, 19.336717229692276, 4.713727777016969, -14.124166774537755, 19.637671845727457, -28.621711341426796, -29.925579194994736  …  -8.109907686058865, -24.53179753147426, -10.186026764954057, -21.862331031719346, -27.05828224341019, -28.84016416943801, 10.852818020455876, -15.45763470586192, 28.117598380373536, -28.281565276399956]

# Simulación de movimiento ABP con dinámica SIR

In [ ]:
using Random
using CSV
using DataFrames

#Parametros 
n = 400
L = 100
Dr = 0.0225 
Dt = 0.03
v0 = 20 
dt = 0.001 
steps = 5000
tend = steps * dt 
a = 1
e = 3
sigma = 2*a
r_min = 2^(1/6)*sigma
frec_save = 10

incremento_rotacional = sqrt(2 * Dr * dt)
incremento_traslacional = sqrt(2 * Dt * dt)

#Angulos iniciales
theta = 2 * pi * rand(n)
cost = cos.(theta)
sint = sin.(theta)

#Velocidades iniciales
velocidades = [v0 * cost, v0 * sint]

# #Rangos en X e Y
# Xmin = -L / 2
# Xmax = L / 2 
# Ymin = -L / 2
# Ymax = L / 2

#DINAMICA SIR
colores = fill("blue", n)
colores[1] = "red"
R = 1.5*r_min

susceptibles = fill(1, n)
susceptibles[1] = 0
infectados = fill(0, n)
infectados[1] = 1
recuperados = fill(0, n)

tmin = 500
tmax = 750
tiempo_rec = (tmax - tmin) * rand(n) .+ tmin 
contador_tiempo = zeros(n)



"""Funcion de posiciones_iniciales!, LJ_fuerza!, dinamica_ABP!"""
function posiciones_iniciales!(n, sigma, L)
    min = -L/2 + sigma
    max = L/2 - sigma

    posiciones = zeros(n, 2)
    X = 0
    Y = 0

    for i in 1:n
        valid_posicion = false 
        while  !valid_posicion
            X = (max - min) * rand() + min
            Y = (max - min) * rand() + min 
            valid_posicion = true
            
            for j in 1:i
                distance = sqrt((X - posiciones[j, 1])^2 + (Y - posiciones[j, 2])^2)
                if distance < sigma*2.0
                    valid_posicion = false
                    break
                end
            end
        end
        
        posiciones[i,1] = X
        posiciones[i,2] = Y
    end 
    return posiciones 
end


function LJ_fuerza!(e, sigma, X, Y, L, r_min)
    f_x = zeros(n)
    f_y = zeros(n)

    for i in 1:n, j in i+1:n
        dx, dy = X[i] - X[j], Y[i] - Y[j]
        dx -= L * round(dx / L)
        dy -= L * round(dy / L)
        distancia = sqrt(dx^2 + dy^2)

        if distancia < r_min
            force = 4 * e * (12 * (sigma/distancia)^12 - 6 * (sigma/distancia)^6) / distancia^2
            f_x[i] += force * dx
            f_y[i] += force * dy
            f_x[j] -= force * dx
            f_y[j] -= force * dy
        end
    end

    return f_x, f_y
end




#Funcion que actualiza posicion de particulas
function dinamica_ABP!(X, Y, velocidades)
    
    #Cambio en el angulo de las particulas
    @. theta += incremento_rotacional * randn() 

    velocidades[1] .= v0 * cos.(theta)
    velocidades[2] .= v0 * sin.(theta)
    
    
    #calculo de LJ_fuerza
    f_x, f_y = LJ_fuerza!(e, sigma, X, Y, L, r_min)


    #Actualizacion de posiciones
    @. X += velocidades[1] * dt + f_x * dt
    @. Y += velocidades[2] * dt + f_y * dt
    

    #Condiciones de borde periodicas 
    @. X = mod(X + L / 2, L) - L / 2
    @. Y = mod(Y + L / 2, L) - L / 2
    
    # Susceptibles -> Infectados
    for i in 1:n, j in 1:n
        dx, dy = X[i] - X[j], Y[i] - Y[j]
        distancia = sqrt(dx^2 + dy^2)
        if distancia < R && colores[i] == "blue" && colores[j] == "red"
            colores[i] = "red"
            infectados[i] = 1
            susceptibles[i] = 0
        if distancia < R && colores[j] == "blue" && colores[i] == "red"
            colores[j] = "red"
            infectados[j] = 1
            susceptibles[j] = 0 
        end
        end
    end
    
    # Infectados -> Recuperados
    for i in 1:n 
        if colores[i] == "red"
            contador_tiempo[i] += 1 #tiempo desde que fue infectada 
        if colores[i] == "red" && contador_tiempo[i] > tiempo_rec[i]
            colores[i] = "green"
            infectados[i] = 0
            recuperados[i] = 1
            contador_tiempo[i] = 0
        end
        end
    end
    
    return X, Y, colores, susceptibles, infectados, recuperados 
end

posiciones = posiciones_iniciales!(n, sigma, L)
X = posiciones[:, 1]
Y = posiciones[:, 2]


posiciones_x = Vector{Vector{Float64}}()
posiciones_y = Vector{Vector{Float64}}()
color = Vector{Vector{String}}()
historial_susceptibles = Int64[]
historial_infectados = Int64[]
historial_recuperados = Int64[]

@time for i in 1:steps
    dinamica_ABP!(X, Y, velocidades)
    susceptibles_actual = sum(susceptibles)
    infectados_actual = sum(infectados)
    recuperados_actual = sum(recuperados)
    if i == 1 || mod(i, frec_save) == 0
        push!(posiciones_x, copy(X))
        push!(posiciones_y, copy(Y))
        push!(color, copy(colores))
        push!(historial_susceptibles, susceptibles_actual)
        push!(historial_infectados, infectados_actual)
        push!(historial_recuperados, recuperados_actual)
    end
end

CSV.write("posiciones_particulas_SIR.csv", DataFrame(X = posiciones_x, Y = posiciones_y, Colores = color))
CSV.write("Dinamica_SIR.csv", DataFrame(S = historial_susceptibles, I = historial_infectados, R = historial_recuperados))